<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction 


## How to use this notebook
This example runs Sima in locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note bookshould be run in the following way:
1. [Installation](#installation) can be run to install all the relevant tools, like python modules and local worker executable
2. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
3. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
4. [Run analysis](#run) shall be run each time a new analysis needs to be run.
5. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.


### Hints
If you get this message "#ERROR - Error in WorkerHost: Completed, but failed to download one or more files:", please remember the upload common file step.

# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Install  <a id='installation'></a>
Run only when packages and/or local worker  needs to be upgraded. Set _update_packages_ to True first time and when packages should be updated.

In [1]:
update_packages = True
import os
print(os.getcwd())
if update_packages:
    %pip install msa azure-storage-blob cancel-token asyncio httpx dacite stringcase pyjwt overrides python-dateutil pythonnet
    %pip install --index-url https://test.pypi.org/simple/ dnv-onecompute==0.1.0.527 dnv-oneworkflow==0.1.0.527 dnv-sesam-commands==0.1.0.527 --force-reinstall
    %pip install -i https://test.pypi.org/simple/ dnv-sifio --user
    # Python.NET for the seamless integration with the .NET Common Language Runtime (CLR)
    %pip install pythonnet --user
    # QuantConnect for intellisense for the .NET libraries
    %pip install quantconnect-stubs --user

    # SifIO for reading and writing SIF
    %pip install -i https://test.pypi.org/simple/dnv-sifio==0.0.3 --user

    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-onecompute=0.1.0.525
    #! pip install --upgrade --index-url https://test.pypi.org/simple/ dnv-oneworkflow=0.1.0.525
    #! pip install - -upgrade - -index-url https: // test.pypi.org/simple / dnv-sesam-commands = 0.1.0.525
    from oneWorkflowToolBox import *
    await install_workflow_runtime()


c:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://test.pypi.org/simple/
  Using cached https://test-files.pythonhosted.org/packages/58/c7/e34f974f924472c6f203deedb644048068fe1fdbf3f077a91af513ad566d/dnv_onecompute-0.1.0.527-py3-none-any.whl (32 kB)
  Using cached https://test-files.pythonhosted.org/packages/aa/2c/f028b357afa90b10f624cb8d2dfce5edda99a72e41b19725a8c8f1a2ebe7/dnv_oneworkflow-0.1.0.527-py3-none-any.whl (24 kB)
  Using cached https://test-files.pythonhosted.org/packages/d9/56/f5024b6f08175473186bf89d358762d282a7f76f729b5bf82a6002614797/dnv_sesam_commands-0.1.0.527-py3-none-any.whl (6.5 kB)
  Using cached https://test-files.pythonhosted.org/packages/a4/9e/7b2278305feab76061245761000046b9493ce687e97c82550c4e2e6a30d1/msal-1.22.0-py2.py3-none-any.whl (90 kB)
  Using cached https://test-files.pythonhosted.org/packages/3e/84/610f379b46d7d3c2d48eadeed6a12b6d46a43100fea70534f5992d0ac996/azure_storage_blob-2.1.0-py2.py3-none-any.whl (88 kB)
  Using cached https://test-files.pythonhosted.org/packages/89/4

ERROR: Could not find a version that satisfies the requirement asyncio (from dnv-onecompute) (from versions: none)
ERROR: No matching distribution found for asyncio


Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")


Note: you may need to restart the kernel to use updated packages.


INFO: The installed package 'LocalWorkflowRuntime' (version 0.1.0-ci.525) is the latest


## Set up fixed user parameters <a id='fixed'></a>
The below parameters should only be needed to modify once.

In [2]:
import os
#Sima path must be specified
sima_exe_path = r'C:\Program Files\DNV\Sima V4.4-00'         
tempFolderPath = r"c:\OneWorkflowTemp" #local directory for worker host, should be at root due to issues with long file names on Windows
notebook_root_folder = os.getcwd()
print(notebook_root_folder)

c:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra


## Set up custom user parameters <a id='custom'></a>

In [3]:
# local workspace, all results will be put here after local or cloud runs
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace'
# location of common files for all analysis, has to be below workspacePath
workspaceId = "SE28"
loadcase_file = "test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = False


## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [4]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tempFolderPath)#, Platform.Linux, debug=True)
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workspace.common_files_directory
results_folder = workspace.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


The temporary blob storage directory is: c:\OneWorkflowTemp\oc_uoer6lg5_blob
The temporary jobs root directory is: c:\OneWorkflowTemp\oc_cfb90ra9_jobs
c:\OneWorkflowTemp


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [5]:

from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\Floater.ssg
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\postprocessing.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\sestra_template.inp to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\sestra_template.inp
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\SimaTemplate.stask to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\SimaTemplate.stask
Uploading C

## Prepare Sima input  <a id='prepare'></a>


### Create command line input for Sima
The function below will create the command line input that will be used to run Sima. 

In [6]:
from dnv.onecompute import FileSpecification


def get_commands_inputs(
        stask_file: str,
        case: dict[str, Any])->dict[str, dict[str, Any]]:
    commands = dict[str, Any]()
    commands["--consoleLog"] = ""
    commands["--log-level"] = "ALL"
    commands["--data"] = "."
    commands["--import"] = dict(file=FileSpecification(sharedfolder=True,
                                directory=commonfiles_folder, filename=stask_file))
    commands["--run"] = dict(task="WorkflowTask",
                             workflow="ExampleWorkflow")

    return {"commands": commands, "inputs": case}

### Create load cases with unique Sima input <a id='loadcase_id'></a>
This code generates the input needed for Sima for the individual load cases. The input parameters to vary in the Sima run are give in the loadcasefile.

In [7]:
import pandas as pd
from dnv.sesam.sima_command import SimaCommand
from dnv.onecompute.flowmodel import ParallelWork
from dnv.oneworkflow import PythonCommand
def get_parallel_work(single_task: bool = False):
    """Returns a parallel processing unit. If the single_task parameter is set to True, the unit will only contain the first task. 
    If set to False, all tasks will be included. The single task option is primarily used for testing purposes."""
    os.chdir(workspace.workspace_path)
    load_cases_parent_folder_name = workspace.load_cases_parent_directory

    parallel_work = ParallelWork()
    parallel_work.work_items.clear()

    # Open environmental input file
    df_cases = pd.read_excel(os.path.join(workspacePath, loadcase_file), index_col=0)
    for loadcase_folder_name, case in df_cases.iterrows():
        load_case_folder = os.path.join(
            load_cases_parent_folder_name, loadcase_folder_name)
        result_folder_lc = os.path.join(workspace.results_directory, loadcase_folder_name)
        # Get SIMA commands and inputs 
        commands_inputs = get_commands_inputs(
             stask_file,  case.to_dict()
        )
        test_command = PythonCommand(
            directory=commonfiles_folder,
            filename="test.py",
            working_dir=loadcase_folder_name,
        )
        # Create SimaCommand instance
        sima_cmd = SimaCommand(sima_exe_path)
        sima_cmd.commands = commands_inputs["commands"]
        sima_cmd.input = commands_inputs["inputs"]
        sima_cmd.sima_result_files =[
                "*-sima.lis",
                "variable*.inp",
                "*.log",
                "results.tda",
                "results.txt",
                "sima_*.res",
                "sys-sima.dat",
                "sima_*.bin",
                "key_sima_*.txt",
                "sima.*"
            ]
        sima_cmd.working_directory = load_case_folder

        # Add work item to ParallelWork instance
        parallel_work.add(sima_cmd, work_unit_id=loadcase_folder_name+"_ID").output_directory(result_folder_lc,
                                                                                         include_files=["**/sima*","**/*.txt", "**/*.lis", "**/*.log"])
        if single_task == True:
            break
    return parallel_work

# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [8]:
%time
from oneWorkflowToolBox import run_workflow_async
import json

"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

# Create Parallel Work Unit and Job
work_unit = get_parallel_work()
job = workflow_client.create_job(work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    min_size_bytes=0,
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/sima*"])
await run_workflow_async(job, workflow_client, downloadOptions)


Wall time: 0 ns
Error: load cases directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\LoadCases does not exist.
The work item c4f84dad-1b9a-486b-8583-b81afe001cf4 message is 'Created'
The work item test001_ID message is 'Created'
The work item test002_ID message is 'Created'
The work item test003_ID message is 'Created'
The progress of the job is '0%'. The message is ''
The work item test001_ID message is ''
The work item test002_ID message is ''
The work item test003_ID message is ''
The work item test001_ID message is '2023-06-05 09:21:16,378 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test002_ID message is '2023-06-05 09:21:16,390 [main] INFO  SIMALog - Setting log level to ALL - at no.marintek.sima.util.SIMALog.setLogLevel(SIMALog.java:51)'
The work item test003_ID message is '2023-06-05 09:21:16,816 [main] INFO  SIMALog - Setting log level to ALL - at no.

In [ ]:
if False:
    #%%writefile "Workspace/CommonFiles/postprocessing.py"
    #%%writefile f"{notebook_root_folder}\Workspace\CommonFiles\processing.py"

    #https://sesam.dnv.com/dev/api/sifio/
    #https://sesam.dnv.com/download/windows64/sesam_input_interface_format.pdf
    #https://sesam.dnv.com/download/windows64/sesam_results_interface_format.pdf
    from matplotlib import cm
    import matplotlib.pyplot as plt
    import os
    from DNV.Sesam.SifApi.Core import ISifData
    from DNV.Sesam.SifApi.IO import SesamDataFactory
    from System.Collections.Generic import List
    import numpy as np
    #os.chdir(r"C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\Workspace\Results\test001")

    def find_displacements_for_given_nodes_and_loadcase (loadcase :int, nodes : list):
        """
        Reads the number of occurrences of a data type and the size of the established pointer table
        for a datatype.
        """

        reader = SesamDataFactory.CreateReader(".", "R3.SIN")
        reader.CreateModel()
        all_data =[]
        for node in nodes:
            print("node:"+ str(node))
            disp_data = reader.ReadData(
                "RVNODDIS", [loadcase, node])
            node_coordinate = reader.ReadData(
                "GCOORD", [node])
            #convert to Python list
            node_coordinate = [
                coordinate for coordinate in node_coordinate[0].Data]
            disp_data = [disp for disp in disp_data[0].Data]
            print(node_coordinate[2:4])
            print("xdisp: " + str(disp_data[5]))
            print("ydisp: " + str(disp_data[6]))
            print("zdisp: " + str(disp_data[7]))
            print("xcoord: " + str(node_coordinate[2]))
            print("ycoord: " + str(node_coordinate[3]))
            print("zcoord: " + str(node_coordinate[4]))
            node_data = node_coordinate[2:5] + disp_data[5:8]
            all_data.append(node_data)
        np.savetxt(f"postprocessedresultsLC{loadcase}.txt",
                all_data, header="xcoord\t\t\t, \t\t\tycoord\t\t\t,\t\t\tzcoord\t\t\t,\t\t\txdisp\t\t\t,\t\t\tydisp\t\t\t, \t\t\tzdisp\t\t")
    
        reader.Dispose()


    lc = 11
    nodes = range(1,200)
    find_displacements_for_given_nodes_and_loadcase(lc, nodes)
    #os.chdir(notebook_root_folder)
    # data = np.loadtxt(
    #     f"{notebook_root_folder}\\postprocessedresultsLC{lc}.txt")
    # xyz = data[:, 0:3]
    # total_disp = np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
    # plt.plot(data[:,0], total_disp,'*')
    # plt.xlabel("x coordinate")
    # plt.ylabel("total disp")

# Run Wasim and Sestra <a id='runwasim'></a>

In [10]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import Platform
import json
import os
from dnv.sesam.sestra_command import SestraCommand
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]

os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {
    'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue
        
        load_case_result_files_dir = os.path.join(results_directory, casename)
        #For testing
        #python_copy_command = PythonCommand(
        #    directory=commonfiles_folder,
        #    filename="copyfiles.py",
        #    working_dir=load_case_result_files_dir)
        casedict = case.to_dict()
        
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)
print(os.getcwd())
print(workflow_client.workspace_path)

work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)
if not cloud_run:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()
print(workflow_client.temp_path)


downloadOptions = FileOptions(
    min_size_bytes =0,
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
#await run_workflow_async(workflow_client.create_job(work_unit), workflow_client, downloadOptions)
await workflow_client.run_workflow_async(
    work_unit,
    work_item_status_changed=work_item_status_changed_callback(workflow_client, downloadOptions),
    log_job=True
)


C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace
C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\Floater.ssg to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\Floater.ssg
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\postprocessing.py to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\postprocessing.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace\CommonFiles\sestra_template.inp to c:\OneWorkflowTemp\oc_uoer6lg5_blob\SE28\CommonFiles\sestra_template.inp
Uploading C:\Users\kblu\source\repos\imp

'd1a548e8-2562-4c83-9f12-0e0dac1c6f47'

# Post processing 

In [ ]:
# imports
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
for folder in glob.glob(f"{notebook_root_folder}\\Workspace\Results\\*"):
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        test_name = folder.split("\\")[-1]
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)

def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(
    multiplot, resultcase=dataFrames.keys())
interactive_plot
